<a href="https://colab.research.google.com/github/edufsi/Projeto-Desafio/blob/Eduardo/Limpeza_Dados_Cambio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import numpy as np

In [57]:
#Importar dados:
url_commodities = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/UNdata_Export_20240412_201846138.csv"
url_parceiros = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_330_202310_Major%20Trading%20Partners.csv"
url_exportacoes = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_123_202310_Total%20Imports%20Exports%20and%20Balance%20of%20Trade.csv"
url_cambio = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/USD_BRL_Historical_Data.csv"


dados_commodities = pd.read_csv(url_commodities)
dados_parceiros = pd.read_csv(url_parceiros, encoding = "ISO-8859-1")
dados_exportacoes = pd.read_csv(url_exportacoes, encoding = "ISO-8859-1")
dados_cambio = pd.read_csv(url_cambio, sep=';')

In [ ]:
# VISÃO GERAL
dados_cambio.head()


In [ ]:
dados_cambio.info()

In [ ]:
dados_cambio.describe()

In [61]:
# Temos 3 problemas principais nesse dataframe.

In [62]:
# O primeiro problema é que a coluna change % está como objeto, mas deveria ser de números

dados_cambio['Change %'] = dados_cambio['Change %'].apply(lambda x: x.replace('%', '')).astype(np.float64) / 100

In [63]:
# O segundo problema é a coluna de data. A data está formatada como mm/dd/aa. Como a formatação pelo menos segue um padrão, podemos arrumar isso fazendo uma
# função simples e aplicando ela a cada linha, sem necessitar o uso de expressões regulares ou nada mais complexo

# formata_data(mm_dd_aa: string) -> aa_mm_dd: string
  # dada uma data na representação mm/dd/aa retorna a mesma data como aa/mm/dd
def formata_data(string):
  mes = string[0] + string[1]
  dia = string[3] + string[4]
  ano = string[6:]

  return ano + '/' + mes + '/' + dia

dados_cambio['Date'] = dados_cambio['Date'].apply(formata_data)

In [65]:
# Por último, a coluna de volumes tem informações apenas durante um intervalo que vai de 2018 a 2023. E mesmo
# dentro desse intervalo possui alguns buracos. Podemos usar isso pra algo, mas vamos deixar separado para tirar do dataframe principal

mascara = dados_cambio.fillna(0)['Vol.'] != 0

# Vamos tomar todo o intervalo, mesmo que tenha alguns pequenos buracos entre eles. Se formos usar essa informação, veremos o que fazer com os buracos

# O index inicial é o nome da primeira linha do DataFrame em que o valor de volume nao e nulo
index_inicial = dados_cambio[mascara].iloc[0].name

# O index final é o nome da última linha do DataFrame em que o valor de volume não é nulo
index_final = dados_cambio[mascara].iloc[-1].name


dados_cambio_volume = dados_cambio.iloc[index_inicial:index_final]


# Pronto, separados esses dados podemos tirar essa coluna do dataframe principal
dados_cambio.drop('Vol.', axis=1, inplace=True)

In [66]:
dados_cambio_volume

,Date,Price,Open,High,Low,Vol.,Change %
314,2023/01/24,5.1392,5.2080,5.2207,5.1344,102.09K,-0.0131
315,2023/01/23,5.2076,5.2079,5.2207,5.1634,130.45K,0.0000
316,2023/01/20,5.2074,5.1746,5.2400,5.1639,115.59K,0.0064
317,2023/01/19,5.1741,5.1868,5.2562,5.1631,156.95K,-0.0024
318,2023/01/18,5.1863,5.1007,5.1901,5.0655,162.32K,0.0169
...,...,...,...,...,...,...,...
1367,2019/01/09,3.6807,3.7138,3.7155,3.6743,48.59K,-0.0088
1368,2019/01/08,3.7133,3.7373,3.7412,3.7034,49.06K,-0.0060
1369,2019/01/07,3.7356,3.7139,3.7384,3.6898,44.59K,0.0054
1370,2019/01/04,3.7155,3.7593,3.7850,3.7088,56.28K,-0.0109


In [67]:
dados_cambio

,Date,Price,Open,High,Low,Change %
0,2024/04/12,5.1175,5.0922,5.1487,5.0890,0.0051
1,2024/04/11,5.0917,5.0735,5.0931,5.0580,0.0034
2,2024/04/10,5.0746,5.0092,5.0863,4.9970,0.0131
3,2024/04/09,5.0090,5.0281,5.0310,4.9995,-0.0035
4,2024/04/08,5.0268,5.0665,5.0759,5.0247,-0.0076
...,...,...,...,...,...,...
7621,1995/01/06,0.8420,0.8400,0.8450,0.8380,0.0030
7622,1995/01/05,0.8395,0.8445,0.8445,0.8395,-0.0059
7623,1995/01/04,0.8445,0.8425,0.8455,0.8425,0.0006
7624,1995/01/03,0.8440,0.8420,0.8470,0.8420,0.0024
